In [1]:
## Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt
import plotly.express as px

## Import functions made specifically for this project
from capstone_functions import *
from plotly_year_functions import *

In [2]:
## Set options for displaying Pandas DataFrames
pd.options.display.max_columns = 100
pd.options.display.max_rows = 120

In [3]:
## Load in voter registration data for Union County
uc_vreg = pd.read_table('Data/ncvoter90_Jan4.zip', encoding='ISO-8859-1')
uc_vreg.head()

,county_id,county_desc,voter_reg_num,status_cd,voter_status_desc,reason_cd,voter_status_reason_desc,absent_ind,name_prefx_cd,last_name,first_name,middle_name,name_suffix_lbl,res_street_address,res_city_desc,state_cd,zip_code,mail_addr1,mail_addr2,mail_addr3,mail_addr4,mail_city,mail_state,mail_zipcode,full_phone_number,race_code,ethnic_code,party_cd,gender_code,birth_age,birth_state,drivers_lic,registr_dt,precinct_abbrv,precinct_desc,municipality_abbrv,municipality_desc,ward_abbrv,ward_desc,cong_dist_abbrv,super_court_abbrv,judic_dist_abbrv,nc_senate_abbrv,nc_house_abbrv,county_commiss_abbrv,county_commiss_desc,township_abbrv,township_desc,school_dist_abbrv,school_dist_desc,fire_dist_abbrv,fire_dist_desc,water_dist_abbrv,water_dist_desc,sewer_dist_abbrv,sewer_dist_desc,sanit_dist_abbrv,sanit_dist_desc,rescue_dist_abbrv,rescue_dist_desc,munic_dist_abbrv,munic_dist_desc,dist_1_abbrv,dist_1_desc,dist_2_abbrv,dist_2_desc,confidential_ind,birth_year,ncid,vtd_abbrv,vtd_desc
0,90,UNION,346446,A,ACTIVE,AV,VERIFIED,,,AALAEI,ARIO,NaN,NaN,4009 MILLBRIDGE PKWY,WAXHAW,NC,28173.0,4009 MILLBRIDGE PKWY,NaN,NaN,NaN,WAXHAW,NC,28173,NaN,W,NL,UNA,M,47,OC,Y,04/25/2019,020A,020A,5.0,WAXHAW,NaN,NaN,9.0,20B,20C,35.0,68.0,NaN,NaN,NaN,NaN,2.0,SCHOOL #2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,WAXHAW,30.0,30TH PROSECUTORIAL,,,N,1974,EF230133,020A,020A
1,90,UNION,348004,A,ACTIVE,AV,VERIFIED,,,AALAEI,KAVITHA,NaN,NaN,4009 MILLBRIDGE PKWY,WAXHAW,NC,28173.0,4009 MILLBRIDGE PKWY,NaN,NaN,NaN,WAXHAW,NC,28173,NaN,A,NL,DEM,F,47,OC,Y,06/24/2019,020A,020A,5.0,WAXHAW,NaN,NaN,9.0,20B,20C,35.0,68.0,NaN,NaN,NaN,NaN,2.0,SCHOOL #2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,WAXHAW,30.0,30TH PROSECUTORIAL,,,N,1974,EF231096,020A,020A
2,90,UNION,281016,A,ACTIVE,AV,VERIFIED,,,AALBORG,ALEXIS,LEANN,NaN,1001 STEVENS PRIDE CT,INDIAN TRAIL,NC,28079.0,1001 STEVENS PRIDE CT,NaN,NaN,NaN,INDIAN TRAIL,NC,28079,NaN,W,NL,UNA,F,27,IA,Y,11/04/2013,029A,029A,6.0,INDIAN TRAIL,NaN,NaN,9.0,20B,20C,35.0,68.0,NaN,NaN,NaN,NaN,3.0,SCHOOL #3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,INDIAN TRAIL,30.0,30TH PROSECUTORIAL,,,N,1994,AL261840,029A,029A
3,90,UNION,288489,A,ACTIVE,AV,VERIFIED,,,AALBORG,BRYAN,LEE,NaN,1001 STEVENS PRIDE CT,INDIAN TRAIL,NC,28079.0,1001 STEVENS PRIDE CT,NaN,NaN,NaN,INDIAN TRAIL,NC,28079,NaN,W,UN,UNA,M,63,OK,Y,09/17/2014,029A,029A,6.0,INDIAN TRAIL,NaN,NaN,9.0,20B,20C,35.0,68.0,NaN,NaN,NaN,NaN,3.0,SCHOOL #3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,INDIAN TRAIL,30.0,30TH PROSECUTORIAL,,,N,1958,AL178177,029A,029A
4,90,UNION,279308,A,ACTIVE,AV,VERIFIED,,,AALBORG,SHARON,SCHLEENBAKER,NaN,1001 STEVENS PRIDE CT,INDIAN TRAIL,NC,28079.0,1001 STEVENS PRIDE CT,NaN,NaN,NaN,INDIAN TRAIL,NC,28079,NaN,W,NL,REP,F,63,PA,Y,07/24/2013,029A,029A,6.0,INDIAN TRAIL,NaN,NaN,9.0,20B,20C,35.0,68.0,NaN,NaN,NaN,NaN,3.0,SCHOOL #3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,INDIAN TRAIL,30.0,30TH PROSECUTORIAL,,,N,1958,AL193958,029A,029A


In [4]:
## Check metadata for voter registration dataframe
uc_vreg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185008 entries, 0 to 185007
Data columns (total 71 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   county_id                 185008 non-null  int64  
 1   county_desc               185008 non-null  object 
 2   voter_reg_num             185008 non-null  int64  
 3   status_cd                 185008 non-null  object 
 4   voter_status_desc         185008 non-null  object 
 5   reason_cd                 185008 non-null  object 
 6   voter_status_reason_desc  185008 non-null  object 
 7   absent_ind                185008 non-null  object 
 8   name_prefx_cd             185008 non-null  object 
 9   last_name                 185004 non-null  object 
 10  first_name                185005 non-null  object 
 11  middle_name               169910 non-null  object 
 12  name_suffix_lbl           21203 non-null   object 
 13  res_street_address        185008 non-null  o

In [5]:
## Check for duplicate entries in voter registration data
uc_vreg.duplicated().sum()

0

In [6]:
## Check number of unique individuals represented in voter reg records
uc_vreg['ncid'].nunique()

185008

In [7]:
## Recast registr_dt as datetime variable
uc_vreg['registr_dt'] = pd.to_datetime(uc_vreg['registr_dt'])

## Fill null values in birth_state with 'Missing'
uc_vreg['birth_state'].fillna(value='Missing', inplace=True)

In [8]:
## Create new column grouping most infrequent party categories (<5% of voters)
  ## into same group as those who are unaffiliated
uc_vreg['party_grp'] = np.where(uc_vreg['party_cd'].isin(['REP', 'DEM']),
                                uc_vreg['party_cd'].str.title(), 'Other')

## Check distribution
check_unique('party_grp', uc_vreg)

,count,%
Rep,75792,40.970000
Other,63647,34.400000
Dem,45569,24.630000


In [9]:
## Create new column grouping most infrequent race categories (<5% of voters)
uc_vreg['race_grp'] = np.where(uc_vreg['race_code'].isin(['W', 'B', 'U']),
                               uc_vreg['race_code'], 'O')
race_grp_map = {'W': 'White',
                'B': 'Black',
                'U': 'Undesig.',
                'O': 'Other'}
uc_vreg['race_grp'] = uc_vreg['race_grp'].map(race_grp_map)

## Check distribution
check_unique('race_grp', uc_vreg)

,count,%
White,132566,71.650000
Black,21357,11.540000
Undesig.,20129,10.880000
Other,10956,5.920000


In [10]:
## Create new column grouping most infrequent cities (<5% of voters)
uc_vreg['city_grp'] = np.where(uc_vreg['res_city_desc'].isin(['MONROE', 
                                                              'WAXHAW',
                                                              'INDIAN TRAIL',
                                                              'MATTHEWS']),
                                uc_vreg['res_city_desc'].str.title(), 'Other')

## Check distribution
check_unique('city_grp', uc_vreg)

,count,%
Monroe,53921,29.150000
Waxhaw,48440,26.180000
Indian Trail,28765,15.550000
Other,27616,14.930000
Matthews,26266,14.200000


In [11]:
## Create a function for grouping birth_state into U.S. Census regions, 
  ## also lumping territories and out of country into 'Other'
def get_birth_reg_census_other(state):
    
    if state in ['AS', 'GU', 'MP', 'PR', 'VI', 'OC']:
        return 'Other'
    
    # the rest of the categories are based on U.S. Census Bureau regions
    elif state in ['CT', 'ME', 'MA', 'NH', 'RI', 'VT',
                     'NJ', 'NY', 'PA']:
        return 'Northeast'
    
    elif state in ['DE', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 
                     'DC', 'WV', 'AL', 'KY', 'MS', 'TN', 'AR',
                     'LA', 'OK', 'TX']:
        return 'South'
    
    elif state in ['IL', 'IN', 'MI', 'OH', 'WI',
                     'IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD']:
        return 'Midwest'
    
    elif state in ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT',
                     'WY', 'AK', 'CA', 'HI', 'OR', 'WA']:
        return 'West'
    
    else:
        return 'Missing'

## Create a new column grouping birth_state into U.S. Census regions,
  ## this time lumping territories and out of country into 'Other'
uc_vreg['birth_reg_other'] = uc_vreg['birth_state'].apply(get_birth_reg_census_other)
check_unique('birth_reg_other', uc_vreg)

,count,%
South,77247,41.750000
Missing,45847,24.780000
Northeast,28941,15.640000
Midwest,15262,8.250000
Other,11196,6.050000
West,6515,3.520000


In [12]:
## Create a function for grouping birth_year into generations, 
  ## also lumping Silent in with Greatest
def get_gen_grp(birth_year):
    
    if birth_year < 1946:
        return 'Greatest-Silent'
    
    elif (birth_year > 1945) & (birth_year < 1965):
        return 'Boomer'
    
    elif (birth_year > 1964) & (birth_year < 1981):
        return 'GenX'
    
    elif (birth_year > 1980) & (birth_year < 1997):
        return 'Millennial'
    
    elif birth_year > 1996:
        return 'GenZ'

## Create a new column grouping birth_year into generations, 
  ## also lumping Silent in with Greatest
uc_vreg['gen_grp'] = uc_vreg['birth_year'].apply(get_gen_grp)
check_unique('gen_grp', uc_vreg)

,count,%
GenX,57071,30.850000
Boomer,48702,26.320000
Millennial,41700,22.540000
GenZ,21050,11.380000
Greatest-Silent,16485,8.910000


In [14]:
## Reformat voter_status_desc labels
uc_vreg['voter_status_desc'] = np.where(
    uc_vreg['voter_status_desc']=='TEMPORARY',
                                'Temp', uc_vreg['voter_status_desc'].str.title()
)

## Check distribution
check_unique('voter_status_desc', uc_vreg)

,count,%
Active,155227,83.900000
Inactive,14748,7.970000
Removed,13554,7.330000
Denied,1356,0.730000
Temp,123,0.070000


In [17]:
## Check distribution
check_unique('reason_cd', uc_vreg)

,count,%
AV,151844,82.070000
IN,9359,5.060000
RD,6179,3.340000
IU,5434,2.940000
RS,3720,2.010000
AL,2317,1.250000
RM,1569,0.850000
RL,1418,0.770000
DU,1356,0.730000
AN,517,0.280000


In [18]:
uc_vreg = uc_vreg[['voter_status_desc', 'reason_cd', 'city_grp', 
                   'race_grp', 'party_grp', 'gen_grp', 'gender_code', 
                   'birth_age', 'birth_reg_other', 'drivers_lic',
                   'registr_dt']].copy()
uc_vreg.head()

,voter_status_desc,reason_cd,city_grp,race_grp,party_grp,gen_grp,gender_code,birth_age,birth_reg_other,drivers_lic,registr_dt
0,Active,AV,Waxhaw,White,Other,GenX,M,47,Other,Y,2019-04-25
1,Active,AV,Waxhaw,Other,Dem,GenX,F,47,Other,Y,2019-06-24
2,Active,AV,Indian Trail,White,Other,Millennial,F,27,Midwest,Y,2013-11-04
3,Active,AV,Indian Trail,White,Other,Boomer,M,63,South,Y,2014-09-17
4,Active,AV,Indian Trail,White,Rep,Boomer,F,63,Northeast,Y,2013-07-24
